In [2]:
import sys
sys.path.append("../")

In [3]:
from libs.ContentDataSet import ContentDataset
from libs.Lsh import LSH
from models.audio.AudioHashModel import AudioHash

In [3]:
from models.video.VideoHashModel import VideoHash

In [3]:
dataset = ContentDataset("../data/compressed_index/")

In [3]:
audioLSH = LSH(threshold=0.81)
videoLSH = LSH(threshold=0.81)

In [7]:
for i in range(len(dataset.content_list)):
  content = dataset.content_list[i]
  for idj,j in enumerate(content[1][:-1]):
    audioLSH.add(j,content[0],idj)

In [13]:
for i in range(len(dataset.content_list)):
  content = dataset.content_list[i]
  for idj,j in enumerate(content[2][:-1]):
    videoLSH.add(j,content[0],idj)

In [4]:
audioHashes = AudioHash()


In [6]:
videoHashes = VideoHash()

In [6]:
audioHashes.create_table()

NameError: name 'audioHashes' is not defined

In [17]:
videoHashes.create_table()

In [11]:
input = []
for key, value in audioLSH._buckets.items():
  for v in value:
    input.append((key,",".join(map(str, v[0])),v[1],v[2]))
   

In [12]:
audioHashes.insertMany(input)

In [18]:
input = []
for key, value in videoLSH._buckets.items():
  for v in value:
    input.append((key,",".join(map(str, v[0])),v[1],v[2]))

In [19]:
videoHashes.insertMany(input)

In [5]:
h = audioHashes.getMinHash("2151062517600")

In [8]:
f = list(map(int,h[0][0].split(",")))

In [ ]:
minHash = audioLSH.get_min_hash()


In [4]:
valDataset = ContentDataset("../data/compressed_val/")

In [5]:
from db import get_connection
from typing import List, Tuple
class AudioHashh:
  def __init__(self):
    pass

  @classmethod
  def create_table(cls):
    conn = get_connection()
    sql = """
    CREATE TABLE IF NOT EXISTS audio_hash (
        id serial PRIMARY KEY,
        key TEXT NOT NULL,
        minhash TEXT NOT NULL,
        name TEXT NOT NULL,
        frame INTEGER NOT NULL  
      )
    """
    conn.cursor().execute(sql)
    conn.commit()
  
  def insertMany(self, data):
    conn = get_connection()
    sql = "INSERT INTO audio_hash (id, key, minhash, name, frame) VALUES (DEFAULT, %s, %s, %s, %s)"
    conn.cursor().executemany(sql, data)
    conn.commit()
    
  def getMinHash(self, band)->List[Tuple[str,str,str,int]]:
    conn = get_connection()
    sql = """
    SELECT key, minhash, name, frame FROM audio_hash WHERE key in %s
    """
    cur = conn.cursor()
    cur.execute(sql, (band,))
    minhash = cur.fetchall()
    cur.close()
    conn.close()
    return minhash

In [6]:
import numpy as np
import random
from models.audio.AudioHashModel import AudioHash

class MinHash:
  def __init__(self, num_perm):
    self.permutations = self._permutations_generate(num_perm)
  
  def _permutations_generate(self, num_perm, seed=5421)->list:
    np.random.seed(seed)
    permutations = [np.random.choice( 255, 100,replace=False) for _ in range(num_perm)]
    return permutations
      
  def hash(self,array)->list:
    signature = []
    for permutation in self.permutations:
      for j in permutation:
        if(array[j] > 0):
          signature.append(j)
          break
    return signature
  
  def jaccard_similarity(self,signature1, signature2):
    intersection = np.sum(np.bitwise_and(signature1, signature2))
    union = np.sum(np.bitwise_or(signature1, signature2))
    return intersection / union 

class LsH:
    def __init__(self, threshold:float,num_hash_functions:int = 100,bands:int = 25, rows:int = 4):
      assert threshold > 0 and threshold < 1
      assert bands*rows == num_hash_functions
      self._threshold = threshold
      self._num_hash_functions = num_hash_functions
      self._bands = bands
      self._rows = rows
      self._min_hash = MinHash(num_hash_functions)
      self._buckets = {}
    
    def clear(self):
      self._buckets.clear()
    
    def searchInDb(self, fingerprints,audioHashes:AudioHash,name,time):
        min_hashes = self._min_hash.hash(fingerprints)
        band_hashes = []
        compared_sketches = set()
        
        if(len(min_hashes) != self._num_hash_functions):
          return None
        
        
        for i in range(self._bands):
          band_hashes += self._compute_band_hash(min_hashes, i)
        band_hashes = tuple(set(band_hashes))
        print(len(band_hashes),"BANDS")
        dbHashes = audioHashes.getMinHash(band_hashes)
        print(len(dbHashes),"SERACH_RES")
        
        for dbHashTuple in dbHashes:
          if dbHashTuple[0] in compared_sketches:
            continue
          dbHash = list(map(int, dbHashTuple[1].split(",")))
          jac = self._min_hash.jaccard_similarity(dbHash, min_hashes)
          if jac >= self._threshold:
            return (time, dbHashTuple[0], dbHashTuple[2], dbHashTuple[3], jac)
          compared_sketches.add(dbHashTuple[0])
          
        
        # for i in range(self._bands):
        #   band_hashes.append(self._compute_band_hash(min_hashes, i))
        #   hashes = audioHashes.getMinHash(band_hashes[i])
        #   for hash in hashes:
        #     if(band_hashes[i] not in compared_sketches):
        #       hash_to_check = list(map(int, hash[0].split(",")))
        #       jac = self._min_hash.jaccard_similarity(hash_to_check, min_hashes)
        #       if jac >= self._threshold:
        #         return (time, hash[1], hash[2], jac)
        #       compared_sketches.add(band_hashes[i])
            
          # if band_hashes[i] in self._buckets:
          #   for sketch_to_check in self._buckets[band_hashes[i]]:
          #     check_key = ''.join(str(x) for x in sketch_to_check)
          #     if check_key not in compared_sketches:
          #       jac = self._min_hash.jaccard_similarity(sketch_to_check[0], min_hashes)
          #       if jac >= self._threshold:
          #         return (sketch_to_check[0],sketch_to_check[1],sketch_to_check[2], jac)
          #       compared_sketches.add(check_key)
    
    def search(self, fingerprints,name,time, add_to_bucket = True)->tuple :
        min_hashes = self._min_hash.hash(fingerprints)
        band_hashes = []
        compared_sketches = set()
        
        if(len(min_hashes) != self._num_hash_functions):
          return None
        
        
        for i in range(self._bands):
          band_hashes.append(self._compute_band_hash(min_hashes, i))
          if band_hashes[i] in self._buckets:
            for sketch_to_check in self._buckets[band_hashes[i]]:
              check_key = ''.join(str(x) for x in sketch_to_check)
              if check_key not in compared_sketches:
                jac = self._min_hash.jaccard_similarity(sketch_to_check[0], min_hashes)
                if jac >= self._threshold:
                  return (sketch_to_check[0],sketch_to_check[1],sketch_to_check[2], jac)
                compared_sketches.add(check_key)
        if add_to_bucket and len(min_hashes) == self._num_hash_functions:
          for i in range(self._bands):
            if band_hashes[i] not in self._buckets:
              self._buckets[band_hashes[i]] = []
            self._buckets[band_hashes[i]].append((min_hashes, name, time))
            
    def add(self, fingerprints, name, time):
        min_hashes = self._min_hash.hash(fingerprints)
        band_hashes = []
        
        if(len(min_hashes) != self._num_hash_functions):
          return None
        
        for i in range(self._bands):
          band_hashes.append(self._compute_band_hash(min_hashes, i))
          if band_hashes[i] not in self._buckets:
            self._buckets[band_hashes[i]] = []
          self._buckets[band_hashes[i]].append((min_hashes, name, time))
          
    def _compute_band_hash(self, min_hashes: list, i: int) -> str:
        """Compute a hash for quick bucket match search."""
        band_hash_list = []
        for j in range(self._rows):
            # Adding the rows corresponding to ith band
            band_hash_list.append('%02d' % min_hashes[i * self._rows + j])

        # Adding the number i to distinguish between bands
        band_hash_list.append('%02d' % i)
        return ''.join(band_hash_list)
      
    def get_min_hash(self):
      return self._min_hash
      

In [7]:
audioLSH = LsH(threshold=0.79)

In [8]:
audioHashes = AudioHashh()

In [9]:
for content in valDataset.content_list:
  fingerprint = content[1]
  res = []
  for f in fingerprint:
    info = audioLSH.searchInDb(f,audioHashes,content[0],0)
    if(info != None):
      print(info)

START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB SEARCH
FINISH DB SEARCH
START DB S

: 

[]

In [7]:
for haash in h:
  videoLSH.searchInDb(f)

('215,106,25,176,31,252,139,31,101,195,219,222,153,108,69,223,167,21,48,176,32,146,164,137,67,225,165,90,140,215,218,76,16,68,230,92,153,162,149,25,212,213,230,70,235,55,163,167,212,97,147,100,145,98,198,12,30,16,98,205,67,229,217,77,90,179,168,168,140,30,47,154,89,211,28,169,11,224,93,231,151,106,137,26,247,172,140,235,235,56,166,245,72,254,67,231,46,67,216,180',)
('215,106,25,176,31,252,155,31,101,21,219,222,153,108,69,223,177,21,249,176,30,146,27,137,67,225,103,224,104,215,218,244,101,68,14,148,10,22,65,10,244,213,208,244,227,155,11,207,212,247,147,100,221,243,24,12,30,147,158,76,67,24,217,77,249,179,64,100,131,30,134,243,147,211,64,73,11,224,157,47,151,106,137,26,247,246,209,179,156,176,25,245,72,254,67,220,213,60,216,180',)
('215,106,25,176,31,160,139,31,101,21,219,222,153,108,69,223,57,21,54,176,57,34,27,242,87,225,103,90,104,215,218,244,16,68,14,92,153,22,149,183,244,213,208,244,21,55,17,33,212,97,55,100,55,98,37,12,30,16,98,205,139,24,217,58,90,179,156,59,54,30,238,243,86,211,2